<a href="https://colab.research.google.com/github/kristithrasher/Amazon_Vine_Analysis/blob/main/VineReviewAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar



--2022-03-08 01:43:00--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.3’

postgresql-42.2.16. 100%[===================>] 979.38K  5.20MB/s    in 0.2s    

2022-03-08 01:43:01 (5.20 MB/s) - ‘postgresql-42.2.16.jar.3’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# Read in vine data from S3 Buckets
from pyspark import SparkFiles
url ="https://kristithrasher-bucket.s3.amazonaws.com/vine_table.csv"
spark.sparkContext.addFile(url)
vine_df = spark.read.csv(SparkFiles.get("vine_table.csv"), sep=",", header=True, inferSchema=True)


In [35]:
# show the dataframe
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RDIJS7QYB6XNR|          5|            0|          0|   N|                Y|
|R36ED1U38IELG8|          5|            0|          0|   N|                Y|
| R1UE3RPRGCOLD|          2|            1|          1|   N|                Y|
|R298788GS6I901|          5|            0|          0|   N|                Y|
|  RNX4EXOBBPN5|          1|            1|          1|   N|                Y|
|R3BPETL222LMIM|          5|            0|          0|   N|                Y|
|R3SORMPJZO3F2J|          3|            2|          2|   N|                Y|
|R2RDOJQ0WBZCF6|          5|            0|          0|   N|                Y|
|R2B8VBEPB4YEZ7|          5|            0|          0|   N|                Y|
|R1CB783I7B0U52|          1|            0|          1|   N|     

AttributeError: ignored

In [6]:
### DELIVERALBE 2

In [7]:
# Filter the vine_df dataframe to show total_votes greater than or equal to 20
vine_df.filter("total_votes>=20").show()
filtered_vine_df = vine_df.filter("total_votes>=20")
filtered_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RL8D0KJ0J9L0O|          5|          152|        165|   N|                Y|
|R1BEINAIQFBRJC|          5|           21|         23|   N|                Y|
|R2L59KIJH302P9|          4|           26|         26|   N|                Y|
| RR99CPG695T0I|          5|          215|        248|   N|                N|
|R1XQNKKUPCMWVO|          5|           43|         44|   N|                Y|
|R3JUXVCT1NSK2A|          3|           25|         26|   N|                Y|
|R3GNSIFV1J2Y2B|          1|           15|         60|   N|                N|
| RTCRZARYY4LXX|          5|           52|         54|   N|                Y|
|R3OFB4P7Y8WR27|          1|           15|         26|   N|                Y|
|R3MTAYGQM25N63|          4|           58|         59|   N|     

In [8]:
# get info on dataframe
filtered_vine_df.describe()


DataFrame[summary: string, review_id: string, star_rating: string, helpful_votes: string, total_votes: string, vine: string, verified_purchase: string]

In [9]:
filtered_vine_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)



In [10]:
#importing datatypes from pyspark so i can change them
from pyspark.sql.types import DoubleType


In [40]:
# Recast the datatype of the helpful_votes and total_votes from integer to double
new_df = filtered_vine_df.withColumn("helpful_votes",filtered_vine_df.helpful_votes.cast(DoubleType()))
filtered_vine2_df = new_df.withColumn("total_votes",filtered_vine_df.total_votes.cast(DoubleType()))

In [12]:
filtered_vine2_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: double (nullable = true)
 |-- total_votes: double (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)



In [51]:
# Create New DataFrame to retrieve all the rows wehre the number of helpful votes is divided by total votes and >=0.5
from pyspark.sql.functions import col, avg
new_df = filtered_vine2_df.withColumn("calculation", col("helpful_votes") / col("total_votes"))
new_df.filter(new_df.calculation >= 0.5).show()


+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|       calculation|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| RL8D0KJ0J9L0O|          5|        152.0|      165.0|   N|                Y|0.9212121212121213|
|R1BEINAIQFBRJC|          5|         21.0|       23.0|   N|                Y|0.9130434782608695|
|R2L59KIJH302P9|          4|         26.0|       26.0|   N|                Y|               1.0|
| RR99CPG695T0I|          5|        215.0|      248.0|   N|                N|0.8669354838709677|
|R1XQNKKUPCMWVO|          5|         43.0|       44.0|   N|                Y|0.9772727272727273|
|R3JUXVCT1NSK2A|          3|         25.0|       26.0|   N|                Y|0.9615384615384616|
| RTCRZARYY4LXX|          5|         52.0|       54.0|   N|                Y|0.9629629629629629|
|R3OFB4P7Y8WR27|          1|  

In [55]:
# create a new data frame for Paid Vine User reviews
vine_paid_df = new_df.filter("vine == 'Y'")
vine_paid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|       calculation|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R2BQOD1R0228FN|          3|         17.0|       26.0|   Y|                N|0.6538461538461539|
| RC31RUPFOHBHQ|          4|        102.0|      117.0|   Y|                N|0.8717948717948718|
| REN3N1WITLF1Y|          5|         33.0|       37.0|   Y|                N|0.8918918918918919|
| R71RZQ9UZVG47|          4|         38.0|       47.0|   Y|                N|0.8085106382978723|
|R38NMQBH88HLM6|          4|         18.0|       24.0|   Y|                N|              0.75|
|R33FGX9EE3QVR6|          4|         26.0|       26.0|   Y|                N|               1.0|
|R1KIOIK6WEYE59|          3|         19.0|       20.0|   Y|                N|              0.95|
|R25X9BMOB3FD0E|          4|  

In [56]:
# Create a new dataframe for No paid Vine user 
vine_non_paid_df = new_df.filter("vine =='N'")
vine_non_paid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|       calculation|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| RL8D0KJ0J9L0O|          5|        152.0|      165.0|   N|                Y|0.9212121212121213|
|R1BEINAIQFBRJC|          5|         21.0|       23.0|   N|                Y|0.9130434782608695|
|R2L59KIJH302P9|          4|         26.0|       26.0|   N|                Y|               1.0|
| RR99CPG695T0I|          5|        215.0|      248.0|   N|                N|0.8669354838709677|
|R1XQNKKUPCMWVO|          5|         43.0|       44.0|   N|                Y|0.9772727272727273|
|R3JUXVCT1NSK2A|          3|         25.0|       26.0|   N|                Y|0.9615384615384616|
|R3GNSIFV1J2Y2B|          1|         15.0|       60.0|   N|                N|              0.25|
| RTCRZARYY4LXX|          5|  

In [77]:
# Total number of vine paid reviews
vine_paid_reviews = vine_paid_df.count()
print(vine_paid_reviews)

136


In [79]:
# Total number of vine non paid reviews
vine_non_paid_reviews = vine_non_paid_df.count()
print(vine_non_paid_reviews)

18603


In [80]:
# Total number of 5-star reviews for Paid Vine users
total_paid_5_star_reviews = vine_paid_df.filter('star_rating = 5')
total_paid_5_star_reviews.count()

74

In [73]:
# Total number of 5-star reviews for Non Paid Vine users
total_non_paid_5_star_reviews = vine_non_paid_df.filter('star_rating = 5')
total_non_paid_5_star_reviews.count()

8542

In [96]:
from pyspark.sql.functions import col, avg
# Percentage of 5-star Reviews for both paid
percent_five_star_paid_reviews = (total_paid_5_star_reviews.count() / vine_paid_reviews) * 100
percent_five_star_paid_reviews


54.41176470588235

In [98]:
# Percentage of 5-star Revies for unpaid 
percent_five_star_unpaid_reviews = (total_non_paid_5_star_reviews.count() / vine_non_paid_reviews) *100
percent_five_star_unpaid_reviews

45.91732516260818